In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
print("X")

X


In [3]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 104.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 78.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=854a40d56838c64e01b6ac8287bcd8f449b2ac6964c166830629fe436366fed5
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers


In [4]:
from sentence_transformers import SentenceTransformer,util
sentences1 = ["This is an example sentence"]
sentences2 = ["این یک جمله مثال است"]
sentences3 = ["این مثال است"]
sentences4 = ["امروز دوشنبه است"]

model = SentenceTransformer('sentence-transformers/LaBSE')
embeddings1 = model.encode(sentences1)
embeddings2 = model.encode(sentences2)
embeddings3 = model.encode(sentences3)
embeddings4 = model.encode(sentences4)
print(util.cos_sim(embeddings1,embeddings2))
print(util.cos_sim(embeddings1,embeddings3))
print(util.cos_sim(embeddings1,embeddings4))

tensor([[0.9538]])
tensor([[0.8226]])
tensor([[0.4237]])


In [5]:
def align(sentences1,sentences2,window_size):
    scores = {}
    resutls = []

    for i in range(0,len(sentences1)):
        for j in range(i,len(sentences1)):
            for k in range(0,len(sentences2)):
                for l in range(k,len(sentences2)):
                    sents1 = ""
                    sents2 = ""
                    for t in range(i,j+1):
                        sents1 = sents1 + " " +  sentences1[t] 
                    for t in range(k,l+1):
                        sents2 = sents2 + " " +  sentences2[t] 
                    
                   
                    if i not in scores:
                        scores[i]={}
                    if j not in scores[i]:
                        scores[i][j] = {}
                    if k not in scores[i][j]:
                        scores[i][j][k] = {}
                    # if l not in scores[i][j][k]:
                    #     scores[i][j][k][l] = {}
                    if (abs(i-j) < window_size) and (abs(k-l) < window_size):
                      embeddings1 = model.encode([sents1], convert_to_tensor=True)
                      embeddings2 = model.encode([sents2], convert_to_tensor=True)
                      scores[i][j][k][l] =  [util.cos_sim(embeddings1,embeddings2).cpu().numpy()[0][0],"active"]
                      # print("{}-{}-{}-{}".format(i,j,k,l))
                    else:
                      scores[i][j][k][l] = [0, "disactive"]
                    print("{}-{}-{}-{}-{}".format(i,j,k,l,scores[i][j][k][l]))

            
    # print(scores)


    counter = 0

    while True:
    
        t = 0 
        for i in range(0,len(sentences1)):
            for j in range(i,len(sentences1)):
                for k in range(0,len(sentences2)):
                    for l in range(k,len(sentences2)):
                        if t==0 and scores[i][j][k][l][1] == "active":
                            a = i
                            b = j
                            c = k
                            d = l
                            t = 1
                            #print("x-{}-{}-{}-{}".format(a,b,c,d))

                        #print(scores[i][j][k][l][0])
                        if t == 1:
                            if scores[i][j][k][l][0] > scores[a][b][c][d][0] and scores[i][j][k][l][1] == "active":
                                a = i
                                b = j
                                c = k
                                d = l

        
        if t ==0:
            for i in range(0,len(sentences1)):
                for j in range(i,len(sentences1)):
                    for k in range(0,len(sentences2)):
                        for l in range(k,len(sentences2)):
                            if scores[i][j][k][l][1] == "selected":
                                resutls.append([i,j,k,l,scores[i][j][k][l][0]])

            return resutls,scores

        #print("{}-{}-{}-{}".format(a,b,c,d))

        scores[a][b][c][d][1] = "selected"
        #resutls.append([a,b,c,d])

        #print("---")


        for i in range(0,len(sentences1)):
            for j in range(i,len(sentences1)):
                for k in range(0,len(sentences2)):
                    for l in range(k,len(sentences2)):
                        if (i >= a and i <= b) or\
                            (j >= a and j <= b) or\
                            (k >= c and k <= d) or\
                            (l >= c and l <= d) or\
                            (a >= i and a <= j) or\
                            (b >= i and b <= j) or\
                            (c >= k and c <= l) or\
                            (d >= k and d <= l) or\
                            (i<=a and l>=d) or\
                            (i>=b and l<=c):
                                if scores[i][j][k][l][1] == "active" :
                                    scores[i][j][k][l][1] = "disactive"
                                    #print("{}-{}-{}-{}".format(i,j,k,l))

                        
        #print(scores)
        #
        #if counter == 10 : break
        
        
        

        counter += 1

In [6]:
!pip install ipython-autotime
text1 = open("/content/drive/MyDrive/farsi.txt","r").read()
text2 = open("/content/drive/MyDrive/english.txt","r").read()
sentences1 = text1.split("\n")
sentences2 = text2.split("\n")
%load_ext autotime


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.4 MB/s eta 0:00:00
time: 402 µs (started: 2023-04-03 15:54:06 +00:00)


In [7]:
len(sentences1)

11

time: 2.59 ms (started: 2023-04-03 15:54:06 +00:00)


In [8]:
x,y = align(sentences1,sentences2,2)
%load_ext autotime

0-0-0-0-[0.6836698, 'active']
0-0-0-1-[0.6836698, 'active']
0-0-0-2-[0, 'disactive']
0-0-0-3-[0, 'disactive']
0-0-0-4-[0, 'disactive']
0-0-0-5-[0, 'disactive']
0-0-0-6-[0, 'disactive']
0-0-1-1-[0.16735813, 'active']
0-0-1-2-[0.16329977, 'active']
0-0-1-3-[0, 'disactive']
0-0-1-4-[0, 'disactive']
0-0-1-5-[0, 'disactive']
0-0-1-6-[0, 'disactive']
0-0-2-2-[0.14689079, 'active']
0-0-2-3-[0.25366032, 'active']
0-0-2-4-[0, 'disactive']
0-0-2-5-[0, 'disactive']
0-0-2-6-[0, 'disactive']
0-0-3-3-[0.34164804, 'active']
0-0-3-4-[0.42768198, 'active']
0-0-3-5-[0, 'disactive']
0-0-3-6-[0, 'disactive']
0-0-4-4-[0.40958703, 'active']
0-0-4-5-[0.41693664, 'active']
0-0-4-6-[0, 'disactive']
0-0-5-5-[0.42536244, 'active']
0-0-5-6-[0.4456076, 'active']
0-0-6-6-[0.4346833, 'active']
0-1-0-0-[0.7143786, 'active']
0-1-0-1-[0.7143786, 'active']
0-1-0-2-[0, 'disactive']
0-1-0-3-[0, 'disactive']
0-1-0-4-[0, 'disactive']
0-1-0-5-[0, 'disactive']
0-1-0-6-[0, 'disactive']
0-1-1-1-[0.25693378, 'active']
0-1-1-2-[0

time: 10.3 s (started: 2023-04-03 15:54:06 +00:00)


In [9]:
x

[[0, 1, 0, 0, 0.7143786],
 [2, 2, 2, 3, 0.46631122],
 [3, 4, 4, 5, 0.7551185],
 [5, 6, 6, 6, 0.7067863]]

time: 3.32 ms (started: 2023-04-03 15:54:16 +00:00)


In [10]:
sentences1

['رنه دکارت، ۳۱ مارس ۱۵۹۶ میلادی در روستایی از شهرستان اَندر-اِ-لُوار فرانسه زاده\u200cشد.[۲] سیزده\u200cماهه بود که مادرش درگذشت و پدرش، که قاضی و نماینده پارلمان در رِن بِرِتانی بود، بی\u200cدرنگ دوباره ازدواج کرد. سرپرستی و نگهداری از دکارت را مادربزرگ مادری\u200cاش و یک دایه بر دوش گرفتند.[۲]',
 'پدرش او را تشویق نمی\u200cکرد و دستاوردهای دکارت، ارزشی برایش نداشت. پس\u200cاز انتشار Discours de la méthode - نخستین کتاب دکارت که او را به شهرت جهانی رساند، این گفته از پدرش نقل شده است: «میان همه فرزندانم، تنها از این یکی راضی نیستم. مجبور بودم پسری به دنیا بیاورم که چنان مسخره باشد که خود را با پوست گوساله مشغول کند؟» (آدام ۱۹۱۰؛ ۴۳۳ f)[۳]',
 '۱۶۰۴ میلادی، دکارت، که پسری هشت\u200cساله\u200c بود، در لا فِلَش به مدرسه رفت.[۲] این مدرسه را یسوعیان بنا نهاده و در آن، علوم نوین و تعالیم مسیحیت درس می\u200cدادند. دکارت در هشت سال تحصیل در آن مدرسه، ادبیات، منطق، اخلاق، ریاضیات و مابعدالطبیعه فراگرفت. ۱۶۱۱، او در جریان یک سخن\u200cرانی با عنوان اکتشاف چند سیارهٔ سرگردان در اطراف مشتری، از کا

time: 3.48 ms (started: 2023-04-03 15:54:16 +00:00)


In [11]:
sentences2

["René Descartes was born in La Haye en Touraine, Province of Touraine (now Descartes, Indre-et-Loire), France, on 31 March 1596.[25] René Descartes was conceived about halfway through August 1595. His mother, Jeanne Brochard, died a few days after giving birth to a still-born child in May 1597.[26][25] Descartes' father, Joachim, was a member of the Parlement of Brittany at Rennes.[27]:\u200a22\u200a René lived with his grandmother and with his great-uncle. Although the Descartes family was Roman Catholic, the Poitou region was controlled by the Protestant Huguenots.[28] In 1607, late because of his fragile health, he entered the Jesuit Collège Royal Henry-Le-Grand at La Flèche,[29][30] where he was introduced to mathematics and physics, including Galileo's work.[31][32] While there, Descartes first encountered hermetic mysticism. After graduation in 1614, he studied for two years (1615–16) at the University of Poitiers, earning a Baccalauréat and Licence in canon and civil law in 161

time: 3.7 ms (started: 2023-04-03 15:54:16 +00:00)


In [12]:
text3 = open("/content/drive/MyDrive/res3.txt","w")
text = ""
for a in x:
  t = "[{},{}]:[{},{}],{}\n".format(str(a[0]),str(a[1]),str(a[2]),str(a[3]),str(a[4]))
  print(t)
  text += t 
text3.write(t)

[0,1]:[0,0],0.7143786

[2,2]:[2,3],0.46631122

[3,4]:[4,5],0.7551185

[5,6]:[6,6],0.7067863



22

time: 553 ms (started: 2023-04-03 15:54:16 +00:00)


In [13]:
text3 = open("/content/drive/MyDrive/res5.txt","w")
text = ""
for a in x:
  a1 = str(a[0])
  a2 = str(a[2])
  for i in range(a[0]+1,a[1]+1):
    a1 = a1 +  "," + str(i)

  for i in range(a[2]+1,a[3]+1):
    a2 = a2 +  "," + str(i)

  

  t = "[{}]:[{}]\n".format(a1,a2)
  print(t)
  text += t 
print(text)
text3.write(text)
text3.close()

[0,1]:[0]

[2]:[2,3]

[3,4]:[4,5]

[5,6]:[6]

[0,1]:[0]
[2]:[2,3]
[3,4]:[4,5]
[5,6]:[6]

time: 480 ms (started: 2023-04-03 15:54:17 +00:00)


In [14]:
# open("/content/drive/MyDrive/res.txt","w").write("dfsdf")

time: 288 µs (started: 2023-04-03 15:54:17 +00:00)


In [15]:
import numpy as np
from collections import defaultdict
from ast import literal_eval



def _precision(goldalign, testalign):
    """
    Computes tpstrict, fpstrict, tplax, fplax for gold/test alignments
    """
    tpstrict = 0  # true positive strict counter
    tplax = 0     # true positive lax counter
    fpstrict = 0  # false positive strict counter
    fplax = 0     # false positive lax counter

    # convert to sets, remove alignments empty on both sides
    print(testalign)
    
    testalign = set([(tuple(x), tuple(y)) for x, y in testalign if len(x) or len(y)])
    goldalign = set([(tuple(x), tuple(y)) for x, y in goldalign if len(x) or len(y)])

    # mappings from source test sentence idxs to
    #    target gold sentence idxs for which the source test sentence 
    #    was found in corresponding source gold alignment
    src_id_to_gold_tgt_ids = defaultdict(set)
    for gold_src, gold_tgt in goldalign:
        for gold_src_id in gold_src:
            for gold_tgt_id in gold_tgt:
                src_id_to_gold_tgt_ids[gold_src_id].add(gold_tgt_id)

    for (test_src, test_target) in testalign:
        if (test_src, test_target) == ((), ()):
            continue
        if (test_src, test_target) in goldalign:
            # strict match
            tpstrict += 1
            tplax += 1
        else:
            # For anything with partial gold/test overlap on the source,
            #   see if there is also partial overlap on the gold/test target
            # If so, its a lax match
            target_ids = set()
            for src_test_id in test_src:
                for tgt_id in src_id_to_gold_tgt_ids[src_test_id]:
                    target_ids.add(tgt_id)
            if set(test_target).intersection(target_ids):
                fpstrict += 1
                tplax += 1
            else:
                fpstrict += 1
                fplax += 1

    return np.array([tpstrict, fpstrict, tplax, fplax], dtype=np.int32)


def score_multiple(gold_list, test_list, value_for_div_by_0=0.0):
    # accumulate counts for all gold/test files
    pcounts = np.array([0, 0, 0, 0], dtype=np.int32)
    rcounts = np.array([0, 0, 0, 0], dtype=np.int32)
    for goldalign, testalign in zip(gold_list, test_list):
        pcounts += _precision(goldalign=goldalign, testalign=testalign)
        # recall is precision with no insertion/deletion and swap args
        test_no_del = [(x, y) for x, y in testalign if len(x) and len(y)]
        gold_no_del = [(x, y) for x, y in goldalign if len(x) and len(y)]
        rcounts += _precision(goldalign=test_no_del, testalign=gold_no_del)

    # Compute results
    # pcounts: tpstrict,fnstrict,tplax,fnlax
    # rcounts: tpstrict,fpstrict,tplax,fplax

    if pcounts[0] + pcounts[1] == 0:
        pstrict = value_for_div_by_0
    else:
        pstrict = pcounts[0] / float(pcounts[0] + pcounts[1])

    if pcounts[2] + pcounts[3] == 0:
        plax = value_for_div_by_0
    else:
        plax = pcounts[2] / float(pcounts[2] + pcounts[3])

    if rcounts[0] + rcounts[1] == 0:
        rstrict = value_for_div_by_0
    else:
        rstrict = rcounts[0] / float(rcounts[0] + rcounts[1])

    if rcounts[2] + rcounts[3] == 0:
        rlax = value_for_div_by_0
    else:
        rlax = rcounts[2] / float(rcounts[2] + rcounts[3])

    if (pstrict + rstrict) == 0:
        fstrict = value_for_div_by_0
    else:
        fstrict = 2 * (pstrict * rstrict) / (pstrict + rstrict)

    if (plax + rlax) == 0:
        flax = value_for_div_by_0
    else:
        flax = 2 * (plax * rlax) / (plax + rlax)

    result = dict(recall_strict=rstrict,
                  recall_lax=rlax,
                  precision_strict=pstrict,
                  precision_lax=plax,
                  f1_strict=fstrict,
                  f1_lax=flax)

    return result

def read_alignments(infile):
    alignments = []
    for line in infile:
            fields = [x.strip() for x in line.split(':') if len(x.strip())]
            if len(fields) < 2:
                raise Exception('Got line "%s", which does not have at least two ":" separated fields' % line.strip())
            try:
                src = literal_eval(fields[0])
                tgt = literal_eval(fields[1])
            except:
                raise Exception('Failed to parse line "%s"' % line.strip())
            alignments.append((src, tgt))

    # I know bluealign files have a few entries entries missing,
    #   but I don't fix them in order to be consistent previous reported scores
    return alignments


time: 2.21 ms (started: 2023-04-03 15:54:17 +00:00)


In [16]:
# golds = open("/content/drive/MyDrive/gold.txt","r").read().splitlines()

# aligns = open("/content/drive/MyDrive/res2.txt","r").read().splitlines()

time: 267 µs (started: 2023-04-03 15:54:17 +00:00)


In [17]:
# golds

time: 262 µs (started: 2023-04-03 15:54:17 +00:00)


In [18]:
# aligns

time: 360 µs (started: 2023-04-03 15:54:17 +00:00)


In [19]:
# score_multiple([read_alignments(golds)],[read_alignments(golds)])

time: 296 µs (started: 2023-04-03 15:54:17 +00:00)


In [20]:
"""
Copyright 2019 Brian Thompson

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
"""

import logging
import sys
from ast import literal_eval
from collections import OrderedDict
from math import ceil
from time import time

import numpy as np

import pyximport
pyximport.install(setup_args={'include_dirs':np.get_include()}, inplace=True, reload_support=True)

from drive.MyDrive.dp_core import make_dense_costs, score_path, sparse_dp, make_sparse_costs, dense_dp

logger = logging.getLogger('vecalign')  # set up in vecalign.py


def preprocess_line(line):
    line = line.strip()
    if len(line) == 0:
        line = 'BLANK_LINE'
    return line


def yield_overlaps(lines, num_overlaps):
    lines = [preprocess_line(line) for line in lines]
    for overlap in range(1, num_overlaps + 1):
        for out_line in layer(lines, overlap):
            # check must be here so all outputs are unique
            out_line2 = out_line[:10000]  # limit line so dont encode arbitrarily long sentences
            yield out_line2


def read_in_embeddings(text_file, embed_file):
    """
    Given a text file with candidate sentences and a corresponing embedding file,
       make a maping from candidate sentence to embedding index, 
       and a numpy array of the embeddings
    """
    sent2line = dict()
    with open(text_file, 'rt', encoding="utf-8") as fin:
        for ii, line in enumerate(fin):
            if line.strip() in sent2line:
                raise Exception('got multiple embeddings for the same line')
            sent2line[line.strip()] = ii

    line_embeddings = np.fromfile(embed_file, dtype=np.float32, count=-1)
    if line_embeddings.size == 0:
        raise Exception('Got empty embedding file')

    laser_embedding_size = line_embeddings.size // len(sent2line)  # currently hardcoded to 1024
    if laser_embedding_size != 1024:
        logger.warning('expected an embedding size of 1024, got %s', laser_embedding_size)
    logger.info('laser_embedding_size determined to be %d', laser_embedding_size)
    line_embeddings.resize(line_embeddings.shape[0] // laser_embedding_size, laser_embedding_size)
    return sent2line, line_embeddings


def make_doc_embedding(sent2line, line_embeddings, lines, num_overlaps):
    """
    lines: sentences in input document to embed
    sent2line, line_embeddings: precomputed embeddings for lines (and overlaps of lines)
    """

    lines = [preprocess_line(line) for line in lines]

    vecsize = line_embeddings.shape[1]

    vecs0 = np.empty((num_overlaps, len(lines), vecsize), dtype=np.float32)

    for ii, overlap in enumerate(range(1, num_overlaps + 1)):
        for jj, out_line in enumerate(layer(lines, overlap)):
            try:
                line_id = sent2line[out_line]
            except KeyError:
                logger.warning('Failed to find overlap=%d line "%s". Will use random vector.', overlap, out_line)
                line_id = None

            if line_id is not None:
                vec = line_embeddings[line_id]
            else:
                vec = np.random.random(vecsize) - 0.5
                vec = vec / np.linalg.norm(vec)

            vecs0[ii, jj, :] = vec

    return vecs0


def make_norm1(vecs0):
    """
    make vectors norm==1 so that cosine distance can be computed via dot product
    """
    for ii in range(vecs0.shape[0]):
        for jj in range(vecs0.shape[1]):
            norm = np.sqrt(np.square(vecs0[ii, jj, :]).sum())
            vecs0[ii, jj, :] = vecs0[ii, jj, :] / (norm + 1e-5)


def layer(lines, num_overlaps, comb=' '):
    """
    make front-padded overlapping sentences
    """
    if num_overlaps < 1:
        raise Exception('num_overlaps must be >= 1')
    out = ['PAD', ] * min(num_overlaps - 1, len(lines))
    for ii in range(len(lines) - num_overlaps + 1):
        out.append(comb.join(lines[ii:ii + num_overlaps]))
    return out


def read_alignments(fin):
    alignments = []
    with open(fin, 'rt', encoding="utf-8") as infile:
        for line in infile:
            fields = [x.strip() for x in line.split(':') if len(x.strip())]
            if len(fields) < 2:
                raise Exception('Got line "%s", which does not have at least two ":" separated fields' % line.strip())
            try:
                src = literal_eval(fields[0])
                tgt = literal_eval(fields[1])
            except:
                raise Exception('Failed to parse line "%s"' % line.strip())
            alignments.append((src, tgt))

    # I know bluealign files have a few entries entries missing,
    #   but I don't fix them in order to be consistent previous reported scores
    return alignments


def print_alignments(alignments, scores=None, file=sys.stdout):
    if scores is not None:
        for (x, y), s in zip(alignments, scores):
            print('%s:%s:%.6f' % (x, y, s), file=file)
    else:
        for x, y in alignments:
            print('%s:%s' % (x, y), file=file)


class DeletionKnob(object):
    """
    A good deletion penalty is dependent on normalization, and probably language, domain, etc, etc
    I want a way to control deletion penalty that generalizes well...
    Sampling costs and use percentile seems to work fairly well.
    """
    def __init__(self, samp, res_min, res_max):

        self.res_min = res_min
        self.res_max = res_max

        if self.res_min >= self.res_max:
            logger.warning('res_max <= res_min, increasing it')
            self.res_max = self.res_min + 1e-4

        num_bins = 1000
        num_pts = 30

        self.hist, self.bin_edges = np.histogram(samp, bins=num_bins,
                                                 range=[self.res_min, self.res_max],
                                                 density=True)

        dx = self.bin_edges[1] - self.bin_edges[0]
        self.cdf = np.cumsum(self.hist) * dx

        interp_points = [(0, self.res_min), ]
        for knob_val in np.linspace(0, 1, num_pts - 1)[1:-1]:
            cdf_idx = np.searchsorted(self.cdf, knob_val)
            cdf_val = self.res_min + cdf_idx / float(num_bins) * (self.res_max - self.res_min)
            interp_points.append((knob_val, cdf_val))
        interp_points.append((1, self.res_max))
        self.x, self.y = zip(*interp_points)

    def percentile_frac_to_del_penalty(self, knob_val):
        del_pen = np.interp([knob_val], self.x, self.y)[0]
        return del_pen


def make_alignment_types(max_alignment_size):
    # return list of all (n,m) where n+m <= this
    alignment_types = []
    for x in range(1, max_alignment_size):
        for y in range(1, max_alignment_size):
            if x + y <= max_alignment_size:
                alignment_types.append((x, y))
    return alignment_types


def ab2xy_w_offset(aa, bb_idx, bb_offset):
    bb_from_side = bb_idx + bb_offset[aa]
    xx = aa - bb_from_side
    yy = bb_from_side
    return (xx, yy)


def xy2ab_w_offset(xx, yy, bb_offset):
    aa = xx + yy
    bb_from_side = yy
    bb = bb_from_side - bb_offset[aa]
    return aa, bb


def process_scores(scores, alignments):
    # floating point sometimes gives negative numbers, which is a little unnerving ...
    scores = np.clip(scores, a_min=0, a_max=None)

    for ii, (x_algn, y_algn) in enumerate(alignments):
        # deletion penalty is pretty arbitrary, just report 0
        if len(x_algn) == 0 or len(y_algn) == 0:
            scores[ii] = 0.0
        # report sores un-normalized by alignment sizes
        #    (still normalized with random vectors, though)
        else:
            scores[ii] = scores[ii] / len(x_algn) / len(y_algn)

    return scores


def sparse_traceback(a_b_csum, a_b_xp, a_b_yp, b_offset, xsize, ysize):
    alignments = []
    xx = xsize
    yy = ysize

    cum_costs = []

    while True:
        aa, bb = xy2ab_w_offset(xx, yy, b_offset)

        cum_costs.append(a_b_csum[aa, bb])

        xp = a_b_xp[aa, bb]
        yp = a_b_yp[aa, bb]

        if xx == yy == 0:
            break

        if xx < 0 or yy < 0:
            raise Exception('traceback bug')

        x_side = list(range(xx - xp, xx))
        y_side = list(range(yy - yp, yy))
        alignments.append((x_side, y_side))

        xx = xx - xp
        yy = yy - yp

    alignments.reverse()
    cum_costs.reverse()
    costs = np.array(cum_costs[1:]) - np.array(cum_costs[:-1])
    # "costs" are scaled by x_alignment_size * y_alignment_size
    #     and the cost of a deletion is del_penalty
    # "scores": 0 for deletion/insertion, 
    #    and cosine distance, *not* scaled 
    #    by len(x_alignment)*len(y_alignment)
    scores = process_scores(scores=costs, alignments=alignments)

    return alignments, scores


def dense_traceback(x_y_tb):
    xsize, ysize = x_y_tb.shape

    xx = xsize - 1
    yy = ysize - 1

    alignments = []
    while True:
        if xx == yy == 0:
            break
        bp = x_y_tb[xx, yy]
        if bp == 0:
            xp, yp = 1, 1
            alignments.append(([xx - 1], [yy - 1]))
        elif bp == 1:
            xp, yp = 0, 1
            alignments.append(([], [yy - 1]))
        elif bp == 2:
            xp, yp = 1, 0
            alignments.append(([xx - 1], []))
        else:
            raise Exception('got unknown value')

        xx = xx - xp
        yy = yy - yp

    alignments.reverse()

    return alignments


def append_slant(path, xwidth, ywidth):
    """
    Append quantized approximation to a straight line
       from current x,y to a point at (x+xwidth, y+ywidth)
    """
    NN = xwidth + ywidth
    xstart, ystart = path[-1]
    for ii in range(1, NN + 1):
        x = xstart + round(xwidth * ii / NN)
        y = ystart + round(ywidth * ii / NN)
        # In the case of ties we want them to round differently,
        #   so explicitly make sure we take a step of 1, not 0 or 2
        lastx, lasty = path[-1]
        delta = x + y - lastx - lasty
        if delta == 1:
            path.append((x, y))
        elif delta == 2:
            path.append((x - 1, y))
        elif delta == 0:
            path.append((x + 1, y))


def alignment_to_search_path(algn):
    """
    Given an alignment, make searchpath.
    Searchpath must step exactly one position in x XOR y at each time step.
    
    In the case of a block of deletions, the order found by DP is not meaningful.
    To make things consistent and to improve the probability of recovering 
       from search errors, we search an approximately straight line
       through a block of deletions. We do the same through a many-many 
       alignment, even though we currently don't refine a many-many alignment...
    """
    path = [(0, 0), ]
    xdel, ydel = 0, 0
    ydel = 0
    for x, y in algn:
        if len(x) and len(y):
            append_slant(path, xdel, ydel)
            xdel, ydel = 0, 0
            append_slant(path, len(x), len(y))
        elif len(x):
            xdel += len(x)
        elif len(y):
            ydel += len(y)

    append_slant(path, xdel, ydel)

    return path


def extend_alignments(course_alignments, size0, size1):
    """
    extend alignments to include new endpoints size0, size1
    if alignments are larger than size0/size1, raise exception
    """
    # could be a string of deletions or insertions at end, so cannot just grab last one
    xmax = 0  # maximum x value in course_alignments
    ymax = 0  # maximum y value in course_alignments
    for x, y in course_alignments:
        for xval in x:
            xmax = max(xmax, xval)
        for yval in y:
            ymax = max(ymax, yval)

    if xmax > size0 or ymax > size1:
        raise Exception('asked to extend alignments but already bigger than requested')

    # do not duplicate xmax/ymax, do include size0/size1 
    extra_x = list(range(xmax + 1, size0 + 1))
    extra_y = list(range(ymax + 1, size1 + 1))

    logger.debug('extending alignments in x by %d and y by %d', len(extra_x), len(extra_y))

    if len(extra_x) == 0:
        for yval in extra_y:
            course_alignments.append(([], [yval]))
    elif len(extra_y) == 0:
        for xval in extra_x:
            course_alignments.append(([xval], []))
    else:
        course_alignments.append((extra_x, extra_y))


def upsample_alignment(algn):
    def upsample_one_alignment(xx):
        return list(range(min(xx) * 2, (max(xx) + 1) * 2))

    new_algn = []
    for xx, yy in algn:
        if len(xx) == 0:
            for yyy in upsample_one_alignment(yy):
                new_algn.append(([], [yyy]))
        elif len(yy) == 0:
            for xxx in upsample_one_alignment(xx):
                new_algn.append(([xxx], []))
        else:
            new_algn.append((upsample_one_alignment(xx), upsample_one_alignment(yy)))
    return new_algn


def make_del_knob(e_laser,
                  f_laser,
                  e_laser_norms,
                  f_laser_norms,
                  sample_size):
    e_size = e_laser.shape[0]
    f_size = f_laser.shape[0]

    if e_size > 0 and f_size > 0 and sample_size > 0:

        if e_size * f_size < sample_size:
            # dont sample, just compute full matrix
            sample_size = e_size * f_size
            x_idxs = np.zeros(sample_size, dtype=np.int32)
            y_idxs = np.zeros(sample_size, dtype=np.int32)
            c = 0
            for ii in range(e_size):
                for jj in range(f_size):
                    x_idxs[c] = ii
                    y_idxs[c] = jj
                    c += 1
        else:
            # get random samples
            x_idxs = np.random.choice(range(e_size), size=sample_size, replace=True).astype(np.int32)
            y_idxs = np.random.choice(range(f_size), size=sample_size, replace=True).astype(np.int32)

        # output
        random_scores = np.empty(sample_size, dtype=np.float32)

        score_path(x_idxs, y_idxs,
                   e_laser_norms, f_laser_norms,
                   e_laser, f_laser,
                   random_scores, )

        min_score = 0
        max_score = max(random_scores)  # could bump this up... but its probably fine

    else:
        # Not much we can do here...
        random_scores = np.array([0.0, 0.5, 1.0])  # ???
        min_score = 0
        max_score = 1  # ????

    del_knob = DeletionKnob(random_scores, min_score, max_score)

    return del_knob


def compute_norms(vecs0, vecs1, num_samples, overlaps_to_use=None):
    # overlaps_to_use = 10  # 10 matches before

    overlaps1, size1, dim = vecs1.shape
    overlaps0, size0, dim0 = vecs0.shape
    assert (dim == dim0)

    if overlaps_to_use is not None:
        if overlaps_to_use > overlaps1:
            raise Exception('Cannot use more overlaps than provided. You may want to re-run make_verlaps.py with a larger -n value')
    else:
        overlaps_to_use = overlaps1

    samps_per_overlap = ceil(num_samples / overlaps_to_use)

    if size1 and samps_per_overlap:
        # sample other size (from all overlaps) to compre to this side
        vecs1_rand_sample = np.empty((samps_per_overlap * overlaps_to_use, dim), dtype=np.float32)
        for overlap_ii in range(overlaps_to_use):
            idxs = np.random.choice(range(size1), size=samps_per_overlap, replace=True)
            random_vecs = vecs1[overlap_ii, idxs, :]
            vecs1_rand_sample[overlap_ii * samps_per_overlap:(overlap_ii + 1) * samps_per_overlap, :] = random_vecs

        norms0 = np.empty((overlaps0, size0), dtype=np.float32)
        for overlap_ii in range(overlaps0):
            e_laser = vecs0[overlap_ii, :, :]
            sim = np.matmul(e_laser, vecs1_rand_sample.T)
            norms0[overlap_ii, :] = 1.0 - sim.mean(axis=1)

    else:  # no samples, no normalization
        norms0 = np.ones((overlaps0, size0)).astype(np.float32)

    return norms0


def downsample_vectors(vecs1):
    a, b, c = vecs1.shape
    half = np.empty((a, b // 2, c), dtype=np.float32)
    for ii in range(a):
        # average consecutive vectors
        for jj in range(0, b - b % 2, 2):
            v1 = vecs1[ii, jj, :]
            v2 = vecs1[ii, jj + 1, :]
            half[ii, jj // 2, :] = v1 + v2
        # compute mean for all vectors
        mean = np.mean(half[ii, :, :], axis=0)
        for jj in range(0, b - b % 2, 2):
            # remove mean
            half[ii, jj // 2, :] = half[ii, jj // 2, :] - mean
    # make vectors norm==1 so dot product is cosine distance
    make_norm1(half)
    return half


def vecalign(vecs0,
             vecs1,
             final_alignment_types,
             del_percentile_frac,
             width_over2,
             max_size_full_dp,
             costs_sample_size,
             num_samps_for_norm,
             norms0=None,
             norms1=None):
    if width_over2 < 3:
        logger.warning('width_over2 was set to %d, which does not make sense. increasing to 3.', width_over2)
        width_over2 = 3

    # make sure input embeddings are norm==1
    make_norm1(vecs0)
    make_norm1(vecs1)

    # save off runtime stats for summary
    runtimes = OrderedDict()

    # Determine stack depth
    s0, s1 = vecs0.shape[1], vecs1.shape[1]
    max_depth = 0
    while s0 * s1 > max_size_full_dp ** 2:
        max_depth += 1
        s0 = s0 // 2
        s1 = s1 // 2

    # init recursion stack
    # depth is 0-based (full size is 0, 1 is half, 2 is quarter, etc)
    stack = {0: {'v0': vecs0, 'v1': vecs1}}

    # downsample sentence vectors
    t0 = time()
    for depth in range(1, max_depth + 1):
        stack[depth] = {'v0': downsample_vectors(stack[depth - 1]['v0']),
                        'v1': downsample_vectors(stack[depth - 1]['v1'])}
    runtimes['Downsample embeddings'] = time() - t0

    # compute norms for all depths, add sizes, add alignment types
    t0 = time()
    for depth in stack:
        stack[depth]['size0'] = stack[depth]['v0'].shape[1]
        stack[depth]['size1'] = stack[depth]['v1'].shape[1]
        stack[depth]['alignment_types'] = final_alignment_types if depth == 0 else [(1, 1)]

        if depth == 0 and norms0 is not None:
            if norms0.shape != vecs0.shape[:2]:
                print('norms0.shape:', norms0.shape)
                print('vecs0.shape[:2]:', vecs0.shape[:2])
                raise Exception('norms0 wrong shape')
            stack[depth]['n0'] = norms0
        else:
            stack[depth]['n0'] = compute_norms(stack[depth]['v0'], stack[depth]['v1'], num_samps_for_norm)

        if depth == 0 and norms1 is not None:
            if norms1.shape != vecs1.shape[:2]:
                print('norms1.shape:', norms1.shape)
                print('vecs1.shape[:2]:', vecs1.shape[:2])
                raise Exception('norms1 wrong shape')
            stack[depth]['n1'] = norms1
        else:
            stack[depth]['n1'] = compute_norms(stack[depth]['v1'], stack[depth]['v0'], num_samps_for_norm)

    runtimes['Normalize embeddings'] = time() - t0

    # Compute deletion penalty for all depths
    t0 = time()
    for depth in stack:
        stack[depth]['del_knob'] = make_del_knob(e_laser=stack[depth]['v0'][0, :, :],
                                                 f_laser=stack[depth]['v1'][0, :, :],
                                                 e_laser_norms=stack[depth]['n0'][0, :],
                                                 f_laser_norms=stack[depth]['n1'][0, :],
                                                 sample_size=costs_sample_size)
        stack[depth]['del_penalty'] = stack[depth]['del_knob'].percentile_frac_to_del_penalty(del_percentile_frac)
        logger.debug('del_penalty at depth %d: %f', depth, stack[depth]['del_penalty'])
    runtimes['Compute deletion penalties'] = time() - t0
    tt = time() - t0
    logger.debug('%d x %d full DP make features: %.6fs (%.3e per dot product)',
                 stack[max_depth]['size0'], stack[max_depth]['size1'], tt,
                 tt / (stack[max_depth]['size0'] + 1e-6) / (stack[max_depth]['size1'] + 1e-6))
    # full DP at maximum recursion depth
    t0 = time()
    stack[max_depth]['costs_1to1'] = make_dense_costs(stack[max_depth]['v0'],
                                                      stack[max_depth]['v1'],
                                                      stack[max_depth]['n0'],
                                                      stack[max_depth]['n1'])

    runtimes['Full DP make features'] = time() - t0
    t0 = time()
    _, stack[max_depth]['x_y_tb'] = dense_dp(stack[max_depth]['costs_1to1'], stack[max_depth]['del_penalty'])
    stack[max_depth]['alignments'] = dense_traceback(stack[max_depth]['x_y_tb'])
    runtimes['Full DP'] = time() - t0

    # upsample the path up to the top resolution
    compute_costs_times = []
    dp_times = []
    upsample_depths = [0, ] if max_depth == 0 else list(reversed(range(0, max_depth)))
    for depth in upsample_depths:
        if max_depth > 0:  # upsample previoius alignment to current resolution
            course_alignments = upsample_alignment(stack[depth + 1]['alignments'])
            # features may have been truncated when downsampleing, so alignment may need extended
            extend_alignments(course_alignments, stack[depth]['size0'], stack[depth]['size1'])  # in-place
        else:  # We did a full size 1-1 search, so search same size with more alignment types
            course_alignments = stack[0]['alignments']

        # convert couse alignments to a searchpath
        stack[depth]['searchpath'] = alignment_to_search_path(course_alignments)

        # compute ccosts for sparse DP
        t0 = time()
        stack[depth]['a_b_costs'], stack[depth]['b_offset'] = make_sparse_costs(stack[depth]['v0'], stack[depth]['v1'],
                                                                                stack[depth]['n0'], stack[depth]['n1'],
                                                                                stack[depth]['searchpath'],
                                                                                stack[depth]['alignment_types'],
                                                                                width_over2)

        tt = time() - t0
        num_dot_products = len(stack[depth]['b_offset']) * len(stack[depth]['alignment_types']) * width_over2 * 2
        logger.debug('%d x %d sparse DP (%d alignment types, %d window) make features: %.6fs (%.3e per dot product)',
                     stack[max_depth]['size0'], stack[max_depth]['size1'],
                     len(stack[depth]['alignment_types']), width_over2 * 2,
                     tt, tt / (num_dot_products + 1e6))

        compute_costs_times.append(time() - t0)
        t0 = time()
        # perform sparse DP
        stack[depth]['a_b_csum'], stack[depth]['a_b_xp'], stack[depth]['a_b_yp'], \
        stack[depth]['new_b_offset'] = sparse_dp(stack[depth]['a_b_costs'], stack[depth]['b_offset'],
                                                 stack[depth]['alignment_types'], stack[depth]['del_penalty'],
                                                 stack[depth]['size0'], stack[depth]['size1'])

        # performace traceback to get alignments and alignment scores
        # for debugging, avoid overwriting stack[depth]['alignments']
        akey = 'final_alignments' if depth == 0 else 'alignments'
        stack[depth][akey], stack[depth]['alignment_scores'] = sparse_traceback(stack[depth]['a_b_csum'],
                                                                                stack[depth]['a_b_xp'],
                                                                                stack[depth]['a_b_yp'],
                                                                                stack[depth]['new_b_offset'],
                                                                                stack[depth]['size0'],
                                                                                stack[depth]['size1'])
        dp_times.append(time() - t0)

    runtimes['Upsample DP compute costs'] = sum(compute_costs_times[:-1])
    runtimes['Upsample DP'] = sum(dp_times[:-1])

    runtimes['Final DP compute costs'] = compute_costs_times[-1]
    runtimes['Final DP'] = dp_times[-1]

    # log time stats
    max_key_str_len = max([len(key) for key in runtimes])
    for key in runtimes:
        if runtimes[key] > 5e-5:
            logger.info(key + ' took ' + '.' * (max_key_str_len + 5 - len(key)) + ('%.4fs' % runtimes[key]).rjust(7))

    return stack

time: 504 ms (started: 2023-04-03 15:54:17 +00:00)


In [21]:
# from dp_utils import yield_overlaps
from sentence_transformers import SentenceTransformer
from math import ceil
from random import seed as seed
import numpy as np
# from dp_utils import make_alignment_types, \
    # read_in_embeddings, make_doc_embedding, vecalign
# from score import score_multiple
from ast import literal_eval



model = SentenceTransformer('sentence-transformers/LaBSE')
#model = SentenceTransformer('/home/mh-jafari/va/LaBSE', device='cuda')
#model = SentenceTransformer('/home/mh-jafari/va/LaBSE')
#model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
#model = SentenceTransformer('sentence-transformers/all-distilroberta-v1')
#model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')
def read_alignments(infile):
    alignments = []
    for line in infile:
            fields = [x.strip() for x in line.split(':') if len(x.strip())]
            if len(fields) < 2:
                raise Exception('Got line "%s", which does not have at least two ":" separated fields' % line.strip())
            try:
                src = literal_eval(fields[0])
                tgt = literal_eval(fields[1])
            except:
                raise Exception('Failed to parse line "%s"' % line.strip())
            alignments.append((src, tgt))

    # I know bluealign files have a few entries entries missing,
    #   but I don't fix them in order to be consistent previous reported scores
    return alignments

def print_alignments(alignments, scores=None):
    res = []
    if scores is not None:
        for (x, y), s in zip(alignments, scores):
            res.append('%s:%s:%.6f' % (x, y, s))
    else:
        for x, y in alignments:
            res.append('%s:%s' % (x, y))
    return res

def overlaps(lines, num_overlaps=50):
    output = set()
    for out_line in yield_overlaps(lines, num_overlaps):
        output.add(out_line)

    # for reproducibility
    output = list(output)
    output.sort()

    return list(output)

def embed_sentences(sentences):
    embeddings = model.encode(sentences)
    return embeddings

def read_in_embeddings(sentences):
    sent2line = dict()
    ol = overlaps(sentences)
    for ii, line in enumerate(ol):
        if line.strip() in sent2line:
            raise Exception('got multiple embeddings for the same line')
        sent2line[line.strip()] = ii

    vecs = embed_sentences(ol)
    return sent2line, vecs

def align(
    source_text,
    target_text,
    gold_alignment=None,
    alignment_max_size=50,
    del_percentile_frac=0.1,
    max_size_full_dp=300,
    costs_sample_size=100000,
    num_samps_for_norm=1000,
    search_buffer_size=5,
):

    src_sent2line, src_line_embeddings = read_in_embeddings(source_text)
    tgt_sent2line, tgt_line_embeddings = read_in_embeddings(target_text)

    width_over2 = ceil(alignment_max_size / 2.0) + search_buffer_size

    test_alignments = []
    stack_list = []

    src_lines = source_text
    vecs0 = make_doc_embedding(src_sent2line, src_line_embeddings,
                                src_lines, alignment_max_size)

    tgt_lines = target_text
    vecs1 = make_doc_embedding(tgt_sent2line, tgt_line_embeddings,
                                tgt_lines, alignment_max_size)

    final_alignment_types = make_alignment_types(alignment_max_size)

    stack = vecalign(vecs0=vecs0,
                        vecs1=vecs1,
                        final_alignment_types=final_alignment_types,
                        del_percentile_frac=del_percentile_frac,
                        width_over2=width_over2,
                        max_size_full_dp=max_size_full_dp,
                        costs_sample_size=costs_sample_size,
                        num_samps_for_norm=num_samps_for_norm)

    # write final alignments to stdout
    out = {}
    out['alignment'] = print_alignments(stack[0]['final_alignments'],
                        stack[0]['alignment_scores'])


    if gold_alignment is not None:
        test_alignments=stack[0]['final_alignments']
        gold_list = read_alignments(gold_alignment)
        res = score_multiple(gold_list=[gold_list], test_list=[test_alignments])
        out['score'] = res
        # log_final_scores(res)

    return out

time: 5.95 s (started: 2023-04-03 15:54:18 +00:00)


In [22]:
t = align(sentences1,sentences2)

time: 2.61 s (started: 2023-04-03 15:54:24 +00:00)


In [23]:
res = t['alignment']

time: 431 µs (started: 2023-04-03 15:54:26 +00:00)


In [24]:
golds = open("/content/drive/MyDrive/gold.txt","r").read().splitlines()

aligns = open("/content/drive/MyDrive/res5.txt","r").read().splitlines()

time: 473 ms (started: 2023-04-03 15:54:26 +00:00)


In [25]:
golds

['[0]:[4]', '[1]:[5,6]', '[2]:[7,8]', '[3]:[9]', '[5,6]:[10]', '[7]:[11]']

time: 2.85 ms (started: 2023-04-03 15:54:27 +00:00)


In [26]:
aligns

['[0,1]:[0]', '[2]:[2,3]', '[3,4]:[4,5]', '[5,6]:[6]']

time: 6.91 ms (started: 2023-04-03 15:54:27 +00:00)


In [27]:
res

['[0, 1, 2]:[0]:0.275917',
 '[]:[1]:0.000000',
 '[3]:[2, 3, 4, 5]:0.329097',
 '[4, 5, 6, 7, 8, 9, 10]:[6]:0.332890']

time: 4.43 ms (started: 2023-04-03 15:54:27 +00:00)


In [28]:
score_multiple([read_alignments(aligns)],[read_alignments(aligns)])

[([0, 1], [0]), ([2], [2, 3]), ([3, 4], [4, 5]), ([5, 6], [6])]
[([0, 1], [0]), ([2], [2, 3]), ([3, 4], [4, 5]), ([5, 6], [6])]


{'recall_strict': 1.0,
 'recall_lax': 1.0,
 'precision_strict': 1.0,
 'precision_lax': 1.0,
 'f1_strict': 1.0,
 'f1_lax': 1.0}

time: 10.5 ms (started: 2023-04-03 15:54:27 +00:00)


In [29]:
score_multiple([read_alignments(res)],[read_alignments(aligns)])

[([0, 1], [0]), ([2], [2, 3]), ([3, 4], [4, 5]), ([5, 6], [6])]
[([0, 1, 2], [0]), ([3], [2, 3, 4, 5]), ([4, 5, 6, 7, 8, 9, 10], [6])]


{'recall_strict': 0.0,
 'recall_lax': 1.0,
 'precision_strict': 0.0,
 'precision_lax': 0.75,
 'f1_strict': 0.0,
 'f1_lax': 0.8571428571428571}

time: 14 ms (started: 2023-04-03 15:54:27 +00:00)
